In [1]:
import pandas as pd
df = pd.read_csv("../Bitext_Sample_Customer_Service_Testing_Dataset.csv")
df.head()

,utterance,intent,category,tags
0,I have a question about cancelling an order,cancel_order,ORDER,B
1,help canceling the order I have made,cancel_order,ORDER,B
2,I do not know how to cancel an order I have made,cancel_order,ORDER,BE
3,where can I cancel the last order I made?,cancel_order,ORDER,BI
4,I don't want the last order I made,cancel_order,ORDER,B


In [2]:
df.columns

Index(['utterance', 'intent', 'category', 'tags'], dtype='object')

In [3]:
df.isnull().sum()

utterance    0
intent       0
category     0
tags         0
dtype: int64

In [4]:
df["utterance"].str.len().describe()

count    818.000000
mean      42.564792
std       10.332719
min        8.000000
25%       36.000000
50%       43.000000
75%       51.000000
max       60.000000
Name: utterance, dtype: float64

#### Step 2 -> Selecting intents and labels

In [5]:
# from the intents taking 5 categories
selected_intents = [
    "change_order",
    "change_shipping_address",
    "check_refund_policy",
    "contact_human_agent",
    "delivery_period"
]


In [6]:
df_filtered = df[df["intent"].isin(selected_intents)].copy()
df_filtered.head()

,utterance,intent,category,tags
25,I have got to change an item of an order,change_order,ORDER,B
26,can you help me removing several items from an...,change_order,ORDER,BIM
27,could you help me change an order I have made?,change_order,ORDER,BIP
28,I cannot change an item of an order,change_order,ORDER,BE
29,problem with adding something,change_order,ORDER,B


In [7]:
df_filtered['intent'].value_counts()

intent
contact_human_agent        42
change_shipping_address    41
change_order               39
delivery_period            38
check_refund_policy        37
Name: count, dtype: int64

In [8]:
df_filtered = df_filtered[["utterance", "intent"]]
df_filtered.rename(columns={"utterance": "text", "intent": "label"},inplace=True)

df_filtered.head()

,text,label
25,I have got to change an item of an order,change_order
26,can you help me removing several items from an...,change_order
27,could you help me change an order I have made?,change_order
28,I cannot change an item of an order,change_order
29,problem with adding something,change_order


In [9]:
# Encoding 
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_filtered["label_id"] = label_encoder.fit_transform(df_filtered["label"])

In [10]:
#checking for null and unique categories
print(df_filtered.isnull().sum())
print(df_filtered["label_id"].nunique())


text        0
label       0
label_id    0
dtype: int64
5


#### Step 3 -> Train/Test Split

In [11]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_filtered,test_size=0.2,random_state=18,stratify=df_filtered["label_id"])

print("Train size:", len(train_df))
print("Test size:", len(test_df))


Train size: 157
Test size: 40


In [12]:
# label distribution
print("Train label distribution:")
print(train_df["label"].value_counts())

print("\nTest label distribution:")
print(test_df["label"].value_counts())


Train label distribution:
label
contact_human_agent        33
change_shipping_address    33
change_order               31
delivery_period            30
check_refund_policy        30
Name: count, dtype: int64

Test label distribution:
label
contact_human_agent        9
delivery_period            8
change_order               8
change_shipping_address    8
check_refund_policy        7
Name: count, dtype: int64


In [13]:
# Converting to HuggingFace Dataset format
from datasets import Dataset

train_dataset = Dataset.from_pandas(
    train_df[["text", "label_id"]],
    preserve_index=False
)

test_dataset = Dataset.from_pandas(
    test_df[["text", "label_id"]],
    preserve_index=False
)


c:\Users\Kalyan\anaconda3\envs\pl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# checking dataset object
print(train_dataset)
print(train_dataset[0])


Dataset({
    features: ['text', 'label_id'],
    num_rows: 157
})
{'text': 'assistance to talk to somebody', 'label_id': 3}


In [15]:
# Rename label column
train_dataset = train_dataset.rename_column("label_id", "labels")
test_dataset = test_dataset.rename_column("label_id", "labels")


In [16]:
train_dataset.features

{'text': Value('string'), 'labels': Value('int64')}

#### Step 4 -> Tokenization with DistilBERT

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


In [18]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )


In [19]:
# Apllying Tokenizer to datasets
tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True
)

tokenized_test = test_dataset.map(
    tokenize_function,
    batched=True
)


Map: 100%|██████████| 40/40 [00:00<00:00, 4906.05 examples/s]


In [20]:
# Removing raw text 
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_test = tokenized_test.remove_columns(["text"])

In [21]:
# Converting to PyTorch
import torch
tokenized_train.set_format("torch")
tokenized_test.set_format("torch")

In [22]:
print(tokenized_train[0])

{'labels': tensor(3), 'input_ids': tensor([ 101, 5375, 2000, 2831, 2000, 8307,  102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0

#### Step 5 -> Loading pretrained model

In [23]:
df_filtered["label"].nunique()

5

In [24]:
from transformers import AutoModelForSequenceClassification

num_labels = df_filtered["label"].nunique()

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
)

model


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [25]:
print(model.classifier.out_features)

5


#### Step 6 -> Defining Training Arguments

In [26]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)


#### Step 7 -> Evaluation metrics

In [27]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="macro")

    return {
        "accuracy": accuracy,
        "f1": f1
    }


#### Step 8 -> Initializing the trainer

In [28]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


C:\Users\Kalyan\AppData\Local\Temp\ipykernel_21588\3348982657.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [29]:
trainer.train()

c:\Users\Kalyan\anaconda3\envs\pl\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.493800,1.359202,0.950000,0.947569
2,1.102000,0.987550,0.975000,0.974902
3,0.879600,0.852831,1.000000,1.000000


c:\Users\Kalyan\anaconda3\envs\pl\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
c:\Users\Kalyan\anaconda3\envs\pl\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
c:\Users\Kalyan\anaconda3\envs\pl\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


TrainOutput(global_step=60, training_loss=1.2153510093688964, metrics={'train_runtime': 223.9107, 'train_samples_per_second': 2.104, 'train_steps_per_second': 0.268, 'total_flos': 15598870698240.0, 'train_loss': 1.2153510093688964, 'epoch': 3.0})

#### Step 9 -> Prediction on test set

In [35]:
predictions = trainer.predict(tokenized_test)

logits = predictions.predictions
y_true = predictions.label_ids
y_pred = logits.argmax(axis=1)


c:\Users\Kalyan\anaconda3\envs\pl\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


In [40]:
id2label = {
    0: "change_order",
    1: "change_shipping_address",
    2: "check_refund_policy",
    3: "contact_human_agent",
    4: "delivery_period"
}


In [42]:
from sklearn.metrics import classification_report

print(classification_report(
    y_true,
    y_pred,
    target_names=df_filtered["label"].unique()
))


                         precision    recall  f1-score   support

           change_order       1.00      1.00      1.00         8
change_shipping_address       1.00      1.00      1.00         8
    check_refund_policy       1.00      1.00      1.00         7
    contact_human_agent       1.00      1.00      1.00         9
        delivery_period       1.00      1.00      1.00         8

               accuracy                           1.00        40
              macro avg       1.00      1.00      1.00        40
           weighted avg       1.00      1.00      1.00        40



In [43]:
from sklearn.metrics import confusion_matrix
import pandas as pd

cm = confusion_matrix(y_true, y_pred)


labels = list(id2label.values())

cm_df = pd.DataFrame(
    cm,
    index=labels,
    columns=labels
)
cm_df

,change_order,change_shipping_address,check_refund_policy,contact_human_agent,delivery_period
change_order,8,0,0,0,0
change_shipping_address,0,8,0,0,0
check_refund_policy,0,0,7,0,0
contact_human_agent,0,0,0,9,0
delivery_period,0,0,0,0,8


#### Manual Predcition

In [44]:
import torch

def predict_intent(text):
    model.eval()
    
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()
    
    return id2label[predicted_class_id]


In [45]:
test_sentences = [
    "I need to modify the items in my recent order",
    "I want to update the delivery address for my order",
    "Can you explain when I am eligible for a refund?",
    "My order hasn't arrived yet, how long does shipping usually take?",
    "I want to speak with a live customer support agent"
]

for s in test_sentences:
    print(f"Sentence: {s}")
    print(f"Predicted intent: {predict_intent(s)}")
    print("-" * 50)


Sentence: I need to modify the items in my recent order
Predicted intent: change_order
--------------------------------------------------
Sentence: I want to update the delivery address for my order
Predicted intent: change_shipping_address
--------------------------------------------------
Sentence: Can you explain when I am eligible for a refund?
Predicted intent: check_refund_policy
--------------------------------------------------
Sentence: My order hasn't arrived yet, how long does shipping usually take?
Predicted intent: delivery_period
--------------------------------------------------
Sentence: I want to speak with a live customer support agent
Predicted intent: contact_human_agent
--------------------------------------------------


In [46]:
SAVE_DIR = "intent_classifier_distilbert"

model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

('intent_classifier_distilbert\\tokenizer_config.json',
 'intent_classifier_distilbert\\special_tokens_map.json',
 'intent_classifier_distilbert\\vocab.txt',
 'intent_classifier_distilbert\\added_tokens.json',
 'intent_classifier_distilbert\\tokenizer.json')

In [47]:
import json

with open(f"{SAVE_DIR}/label_mapping.json", "w") as f:
    json.dump(id2label, f)


In [48]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer_loaded = AutoTokenizer.from_pretrained(SAVE_DIR)
model_loaded = AutoModelForSequenceClassification.from_pretrained(SAVE_DIR)

model_loaded.eval()


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
